In [ ]:
import warnings
warnings.filterwarnings('ignore')

#################
# My notebook issues
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
sys.path.append('home/egupta/.local/bin')

##############

import os
import numpy as np
import scipy.stats as ss
import networkx as nx
import csv 
import numpy as np
import seaborn as sns
from pandas import DataFrame
from scipy.optimize import curve_fit
import matplotlib
from matplotlib import pyplot as plt

%matplotlib inline


##qtensor imports
import qtensor
from qtensor import QAOA_energy
from qtensor import parameter_optimization as popt
import torch

# Importing Graphs

In [ ]:
file3 = open("New110Distinct20NodeRandomGraphs.txt")
file2 = open("New110Distinct20NodeRandomGraphsData.txt")
g_data = list(np.loadtxt(file2).reshape(110,20,4))
mat_list = list(np.loadtxt(file3).reshape(110,20,20))
def mats_to_graphs(mat_list):
  """
  Given list of adjaceny matrix repr of graphs, converts to actual graphs
  """
  g_list = []
  for mat in mat_list:
    arr = np.array(mat)
    G = nx.convert_matrix.from_numpy_matrix(arr)
    g_list.append(G)
  return g_list
g_list = mats_to_graphs(mat_list)


In [ ]:
def find_max_energy(results):
    """
    Given multiple local optima, find the max
    """
    energies = []
    #print(results)
    for i in range(len(results)):
        energies.append(results[i][2])
    return max(energies)

In [ ]:
max_energies = []
for i in range(len(g_data)):
    e = find_max_energy(g_data[i])
    max_energies.append(e)

# Evaluating energy landscapes 

In [ ]:
def even_percent(graph):
  """
  Determines the relative percent of even degree nodes
  """
  nodes = graph.number_of_nodes()
  tot_ev = 0
  for i in range(nodes):
    if graph.degree[i]%2 == 0:
      tot_ev+=1
  return tot_ev/nodes

In [ ]:
%%capture
datas = []
step = 20
graphs = g_list
for k in range(len(graphs)):
    gamma = np.linspace(0, 1, num = step)
    beta = np.linspace(0, 0.5, num = step)
    data = np.zeros((step, step))
    
    max_e = max_energies[k]

    for i in range(step):
      for j in range(step):
        data[step-i-1, j] = (QAOA_energy(graphs[k], gamma = [gamma[i]], beta = [beta[j]]))/max_e
    fig, ax = plt.subplots(figsize = (12, 7))
    title = "Energy landscape of graph indexed "+str(k)
    plt.title(title, fontsize = 18)
    ttl = ax.title
    ttl.set_position([0.5, 1.05])
    sns.heatmap(data, ax = ax)
    plt.show()
    datas.append(data)

In [ ]:
b_file = open("EnergyLandscapes110RandomGraphsOct30(1).txt", "w")
for data_ in datas:
    for row in data_:
        np.savetxt(b_file, row)
b_file.close()

In [ ]:
percents = [i/10 for i in range(0,11,1)]
datas_av = []
graphs = g_list
for p in percents:
    #identify all graphs with that percentage of even degree nodes 
    ind = []
    for g_ind in range(len(graphs)):
        if even_percent(graphs[g_ind]) == p:
            ind.append(g_ind)
    n = len(ind)
    #now create averaged out energy landscapes
    print(ind)
    data_av = np.zeros((step, step))
    for i in range(step):
        for j in range(step):
            ratios =[datas[k][i][j] for k in ind]
            if n !=0:
                data_av[i][j] = sum(ratios)/n
    datas_av.append(data_av)
            

In [ ]:
step

In [ ]:

df = DataFrame(datas_av[1])

fig, axn = plt.subplots(2, 6, sharex=True, sharey=True)
fig.delaxes(axn[1,5]) #The indexing is zero-based here
cbar_ax = fig.add_axes([.91, .3, .03, .4])
cbar_ax.yaxis.label.set_size(20)

for i, ax in enumerate(axn.flat):
    if i !=11:
        sns.heatmap(datas_av[i], ax=ax,
                    cbar=i == 0,
                    cbar_ax=None if i else cbar_ax)
        ax.set_title( str(int(percents[i]*100)) + '%')#'Parity: ' +


In [ ]:
percents[1]

In [ ]:
datas_av[8]

In [ ]:
datas[19]